# Wprowadzenie 
Nasze wyzwanie jest z jednej strony proste, z drugiej strony dość ambitne. 

Jedno z klasycznych "Hello World" świata Big Data polega na zliczaniu wystąpienia słów. Dane wejściowe - plik tekstowy lub strumień tekstu. Dane wynikowe - liczba wystąpień każdego ze słów. Klasyka. 

My zrobimy to samo, jednak naszymi danymi wejściowymi będą... opowiadania Artura Conan Doyla (czyli standard), ale nie w plikach tekstowych, a w formacie PDF (i to już standard nie jest). 

Trudne? Nic bardziej mylnego. Python to mnogość bibliotek o niezliczonej funkcjonalności. 

Prosty przykład...

Pobierz nasze dane wejściowe

In [1]:
import requests
r = requests.get("https://jankiewicz.pl/bigdata/bigdata-sp/cano-pdf.zip", allow_redirects=True)
open('cano-pdf.zip', 'wb').write(r.content)

7830123

Rozpakuj nasz plik

In [2]:
%%sh
unzip cano-pdf.zip

Archive:  cano-pdf.zip
  inflating: cano-pdf/3gab.pdf       
  inflating: cano-pdf/3gar.pdf       
  inflating: cano-pdf/3stu.pdf       
  inflating: cano-pdf/abbe.pdf       
  inflating: cano-pdf/bery.pdf       
  inflating: cano-pdf/blac.pdf       
  inflating: cano-pdf/blan.pdf       
  inflating: cano-pdf/blue.pdf       
  inflating: cano-pdf/bosc.pdf       
  inflating: cano-pdf/bruc.pdf       
  inflating: cano-pdf/card.pdf       
  inflating: cano-pdf/chas.pdf       
  inflating: cano-pdf/copp.pdf       
  inflating: cano-pdf/cree.pdf       
  inflating: cano-pdf/croo.pdf       
  inflating: cano-pdf/danc.pdf       
  inflating: cano-pdf/devi.pdf       
  inflating: cano-pdf/dyin.pdf       
  inflating: cano-pdf/empt.pdf       
  inflating: cano-pdf/engr.pdf       
  inflating: cano-pdf/fina.pdf       
  inflating: cano-pdf/five.pdf       
  inflating: cano-pdf/glor.pdf       
  inflating: cano-pdf/gold.pdf       
  inflating: cano-pdf/gree.pdf       
  inflating: cano-pdf/houn.

Sprawdź czy mamy zainstalowany potrzebny moduł

# PyPDF2

In [3]:
%%sh
pip freeze | grep PyPDF2

PyPDF2==3.0.1


In [4]:
import PyPDF2 
    
# Utwórz obiekt odnoszący się do przykładowego pliku
pdfFileObj = open('cano-pdf/3gab.pdf', 'rb') 
    
# Utwórz obiekt PdfFileReader 
pdfReader = PyPDF2.PdfReader(pdfFileObj) 
    
# To wszystko 
# Zobacz ile ten plik ma stron 
print(len(pdfReader.pages))

10


In [5]:
# Pobierz pierwszą ze stron
pageObj = pdfReader.pages[0]
    
# Dokonaj esktrakcji tekstu, który się na niej znajduje 
print(pageObj.extract_text()) 

The Adventure of the Three Gables
Arthur Conan Doyle


In [6]:
# Nie zapomnij zamknąć nasz obiekt pliku
pdfFileObj.close() 

Proste prawda? 

No to do roboty. W pierwszej kolejności załadujmy dane tam, gdzie będą one mogły być wydajnie odczytywane przez wiele węzłów klastra

# Przygotowanie danych

In [7]:
%%sh
hadoop fs -mkdir -p cano-pdf

In [8]:
%%sh
hadoop fs -put -f cano-pdf/* cano-pdf/

In [9]:
%%sh
hadoop fs -ls cano-pdf

Found 60 items
-rw-r--r--   2 root hadoop      92929 2023-11-30 12:58 cano-pdf/3gab.pdf
-rw-r--r--   2 root hadoop      85278 2023-11-30 12:58 cano-pdf/3gar.pdf
-rw-r--r--   2 root hadoop      82246 2023-11-30 12:58 cano-pdf/3stu.pdf
-rw-r--r--   2 root hadoop     105053 2023-11-30 12:58 cano-pdf/abbe.pdf
-rw-r--r--   2 root hadoop     105315 2023-11-30 12:58 cano-pdf/bery.pdf
-rw-r--r--   2 root hadoop      94016 2023-11-30 12:58 cano-pdf/blac.pdf
-rw-r--r--   2 root hadoop      94094 2023-11-30 12:58 cano-pdf/blan.pdf
-rw-r--r--   2 root hadoop     103941 2023-11-30 12:58 cano-pdf/blue.pdf
-rw-r--r--   2 root hadoop     115124 2023-11-30 12:58 cano-pdf/bosc.pdf
-rw-r--r--   2 root hadoop     115501 2023-11-30 12:58 cano-pdf/bruc.pdf
-rw-r--r--   2 root hadoop     106359 2023-11-30 12:58 cano-pdf/card.pdf
-rw-r--r--   2 root hadoop      89194 2023-11-30 12:58 cano-pdf/chas.pdf
-rw-r--r--   2 root hadoop     111383 2023-11-30 12:58 cano-pdf/copp.pdf
-rw-r--r--   2 root hadoop      8783

Utwórzmy teraz nasz obiekt konteksu (o ile jeszcze nie istnieje)

# Utworzenie obiektu kontekstu

In [10]:
# w przypadku korzystania z kernela Python
from pyspark import SparkContext, SparkConf

In [11]:
# w przypadku korzystania z kernela Python
conf = SparkConf().setAppName("Spark - RDD - warsztaty").setMaster("yarn")
sc = SparkContext(conf=conf)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/30 12:58:39 INFO SparkEnv: Registering MapOutputTracker
23/11/30 12:58:39 INFO SparkEnv: Registering BlockManagerMaster
23/11/30 12:58:39 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
23/11/30 12:58:39 INFO SparkEnv: Registering OutputCommitCoordinator


Do tej pory szło gładko. Teraz mamy mały problem. <br> 
W jaki sposób zaczytać nasze pliki? 

Nie są to pliki tekstowe, więc `textFile` prowadzający dane linia po linii do naszych dokumentów nie jest tu przydatny.<br>
Zaglądnij na https://spark.apache.org/docs/latest/rdd-programming-guide.html#external-datasets

Właściwie, żadna z metod nie jest tu odpowiednia. 

Zrobimy zatem tak, naszymi danymi wejściowymi nie będą pliki. Będą ich nazwy, a Spark na podstawie tych nazw będzie je odczytywał i ... 

# Przygotowanie metadanych wejściowych

In [12]:
sc

<SparkContext master=yarn appName=Spark - RDD - warsztaty>

In [13]:
%%sh
hadoop fs -ls cano-pdf > files.txt

In [14]:
%%sh
hadoop fs -copyFromLocal files.txt

In [15]:
rawFiles = sc.textFile("files.txt")

In [16]:
rawFiles.collect()

['Found 60 items',
 '-rw-r--r--   2 root hadoop      92929 2023-11-30 12:58 cano-pdf/3gab.pdf',
 '-rw-r--r--   2 root hadoop      85278 2023-11-30 12:58 cano-pdf/3gar.pdf',
 '-rw-r--r--   2 root hadoop      82246 2023-11-30 12:58 cano-pdf/3stu.pdf',
 '-rw-r--r--   2 root hadoop     105053 2023-11-30 12:58 cano-pdf/abbe.pdf',
 '-rw-r--r--   2 root hadoop     105315 2023-11-30 12:58 cano-pdf/bery.pdf',
 '-rw-r--r--   2 root hadoop      94016 2023-11-30 12:58 cano-pdf/blac.pdf',
 '-rw-r--r--   2 root hadoop      94094 2023-11-30 12:58 cano-pdf/blan.pdf',
 '-rw-r--r--   2 root hadoop     103941 2023-11-30 12:58 cano-pdf/blue.pdf',
 '-rw-r--r--   2 root hadoop     115124 2023-11-30 12:58 cano-pdf/bosc.pdf',
 '-rw-r--r--   2 root hadoop     115501 2023-11-30 12:58 cano-pdf/bruc.pdf',
 '-rw-r--r--   2 root hadoop     106359 2023-11-30 12:58 cano-pdf/card.pdf',
 '-rw-r--r--   2 root hadoop      89194 2023-11-30 12:58 cano-pdf/chas.pdf',
 '-rw-r--r--   2 root hadoop     111383 2023-11-30 12:58 

Jesteśmy zainteresowani tylko nazwami plików, a zatem...

In [17]:
import re
rawFiles.filter(lambda s: "cano" in s).map(lambda s: re.search(".* (\S*)$",s).group(1)).collect()

['cano-pdf/3gab.pdf',
 'cano-pdf/3gar.pdf',
 'cano-pdf/3stu.pdf',
 'cano-pdf/abbe.pdf',
 'cano-pdf/bery.pdf',
 'cano-pdf/blac.pdf',
 'cano-pdf/blan.pdf',
 'cano-pdf/blue.pdf',
 'cano-pdf/bosc.pdf',
 'cano-pdf/bruc.pdf',
 'cano-pdf/card.pdf',
 'cano-pdf/chas.pdf',
 'cano-pdf/copp.pdf',
 'cano-pdf/cree.pdf',
 'cano-pdf/croo.pdf',
 'cano-pdf/danc.pdf',
 'cano-pdf/devi.pdf',
 'cano-pdf/dyin.pdf',
 'cano-pdf/empt.pdf',
 'cano-pdf/engr.pdf',
 'cano-pdf/fina.pdf',
 'cano-pdf/five.pdf',
 'cano-pdf/glor.pdf',
 'cano-pdf/gold.pdf',
 'cano-pdf/gree.pdf',
 'cano-pdf/houn.pdf',
 'cano-pdf/iden.pdf',
 'cano-pdf/illu.pdf',
 'cano-pdf/lady.pdf',
 'cano-pdf/last.pdf',
 'cano-pdf/lion.pdf',
 'cano-pdf/maza.pdf',
 'cano-pdf/miss.pdf',
 'cano-pdf/musg.pdf',
 'cano-pdf/nava.pdf',
 'cano-pdf/nobl.pdf',
 'cano-pdf/norw.pdf',
 'cano-pdf/prio.pdf',
 'cano-pdf/redc.pdf',
 'cano-pdf/redh.pdf',
 'cano-pdf/reig.pdf',
 'cano-pdf/resi.pdf',
 'cano-pdf/reti.pdf',
 'cano-pdf/scan.pdf',
 'cano-pdf/seco.pdf',
 'cano-pdf

In [18]:
fileNames = rawFiles.filter(lambda s: "cano" in s).map(lambda s: re.search(".* (\S*)$",s).group(1))

Nie chcemy aby całą ekstrakcję danych tekstowych z plików PDF wykonywał jeden węzeł. Sprawdźmy ile mamy partycji naszego RDD. 

Jeśli będzie ich zbyt mało, możemy zmienić ich liczbę za pomoca metody `repartition(liczba_partycji)`.

Przeanalizuj to ile zasobów ma nasz klaster, w szczególności zwróć uwagę na liczbę procesorów we wszystkich maszynach.

Stosując *regułę kciuka* ustaw liczbę partycji na taką, która jest równa liczbie procesorów. Wprowadź zmiany w powyższej linii, tak aby poniższa potwierdziła oczekiwaną liczbę partycji. 

In [19]:
fileNames.getNumPartitions()

2

# Konwersja metadanych na dane 

Jeśli liczba partycji jest już w porządku, to czas na kluczowy moment. <br>
Chcemy, aby każdy z elementów naszego RDD zamienił się z nazwy pliku, na szereg elementów odnoszących się do poszczególnych linii zawartych w tym pliku. 

Potrzebujemy zatem funkcji, która:
* odczyta plik o podanej nazwie 
* dokona ekstracji jego zawartości
* utworzy listę zawierającą poszczególne linie

Funkcję tą wykorzystamy następnie w metodzie `flatMap` na naszym `RDD`. <br>
Reszta będzie *easy peasy*. 

**Uwaga!** <br>
Plik nie będzie znajdował się w lokalnym systemie plików węzła roboczego... będzie znajdował się w systemie plików HDFS!

Aby sobie z tym poradzić, sprawdźmy czy mamy dostępną jeszcze jedną bibliotekę.

In [20]:
%%sh
pip freeze | grep pydoop

pydoop==2.0.0


In [25]:
def pdf2txt(fileName):
    
    import PyPDF2
    import pydoop.hdfs as hdfs

    # Utwórz obiekt odnoszący się do przykładowego pliku
    pdfFileObj = hdfs.open(fileName, "rb") 
    
    # Utwórz obiekt PdfFileReader 
    pdfReader = PyPDF2.PdfReader(pdfFileObj) 
    
    lines = []
    
    for page in range(len(pdfReader.pages)): 
        pageObj = pdfReader.pages[page] 
        content = pageObj.extract_text() 
        lines.extend(content.splitlines())
    pdfFileObj.close()
    
    return lines

Sprawdźmy ją. Tym razem będzie to odczyt z systemu plików HDFS.

In [26]:
lines_3gab = pdf2txt("cano-pdf/3gab.pdf")

In [27]:
lines_3gab[:3]

['The Adventure of the Three Gables',
 'Arthur Conan Doyle',
 'This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the']

Pozostało nam z niej skorzystać.

In [28]:
lines = fileNames.flatMap(lambda fn: pdf2txt(fn))

Próba generalna

In [29]:
lines.take(2)

['The Adventure of the Three Gables', 'Arthur Conan Doyle']

# Zadania 

Teraz już z górki. Reszta należy do Ciebie. 

**Uwaga!** Na wynikowym RDD, który powinien zawierać dla każdego słowa liczbę jego wystąpień, będziemy wykonywali wiele operacji. <br>
Zadbaj o to, aby każdorazowe użycie tego wynikowego RDD nie powodowało odczytywania plików PDF.

## Zadanie 1

Utwórz obiekt RDD `wordCounts`, który dla każdego słowa liczbę jego wystąpień.

In [32]:
import re
words = lines.flatMap(lambda line: re.split(r'\W+', line))

In [41]:
wordCounts = words.groupBy(lambda word: word).map(lambda pair: (pair[0], len(pair[1])))
print(wordCounts.first())

('The', 2769)


## Zadanie 2

Znajdź 10 najczęściej wykorzystywanych słów.

In [42]:
wordCounts.sortBy(lambda pair: pair[1], False).take(10)

[('', 42232),
 ('the', 33333),
 ('I', 17288),
 ('and', 16818),
 ('of', 16688),
 ('to', 16038),
 ('a', 15083),
 ('that', 10693),
 ('in', 10359),
 ('was', 9763)]

## Zadanie 3

Znajdź 10 najczęściej wykorzystywanych słów, które składają się z co najmniej 5 liter. 

In [47]:
wordCounts.filter(lambda pair: len(pair[0])>=5).sortBy(lambda pair: pair[1], False).take(10)

[('which', 4243),
 ('Holmes', 3044),
 ('there', 2186),
 ('would', 2150),
 ('could', 1843),
 ('should', 1229),
 ('There', 1200),
 ('about', 1096),
 ('before', 1016),
 ('little', 1009)]

## Zadanie 4

Ile razy pojawiło się słowo "Watson"?

In [48]:
wordCounts.filter(lambda pair: pair[0] == "Watson").collect()

[('Watson', 984)]

## Zadanie 5

A ile razy pojawiło się słowo "Moriarty"?

In [49]:
wordCounts.filter(lambda pair: pair[0] == "Moriarty").collect()

[('Moriarty', 49)]